# Homework: Open-Source LLMs

In this homework, we'll experiment more with Ollama
| 

    It's possible that your answers won't match exactly. If it's the case, select the closest one.
## Q1. Running Ollama with Docker

-----
Let's run ollama with Docker. We will need to execute the same command as in the lectures:

In [ ]:
!docker run -it \
    --rm \
    -v ollama:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama

Unable to find image 'ollama/ollama:latest' locally
latest: Pulling from ollama/ollama

c8da3324: Pulling fs layer 
0ab4fb75: Pulling fs layer 
7d737fbb: Pull complete 7.6MB/387.6MBBDigest: sha256:4a3c5b5261f325580d7f4f6440e5094d807784f0513439dcabfda9c2bdf4191e
Status: Downloaded newer image for ollama/ollama:latest
Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIPRZaCnmYa0SCjXptUo46THyPHaZEnkucctNEB1o5CMn

2024/07/02 18:53:33 routes.go:1064: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://0.0.0.0:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE: OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:1 OLLAMA_MAX_QUEUE:512 OLLAMA_MAX_VRAM:0 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://

What's the version of ollama client?

To find out, enter the container and execute ollama with the -v flag.

In [4]:
pwd

'/workspaces/llm-zoomcamp/02-open-source'

In [3]:
cd /root/.ollama/models/manifests/registry.ollama.ai/library


[Errno 13] Permission denied: '/root/.ollama/models/manifests/registry.ollama.ai/library'
/workspaces/llm-zoomcamp/02-open-source


/home/codespace/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


## Q2. Downloading an LLM

--------

We will donwload a smaller LLM - gemma:2b.

Again let's enter the container and pull the model:

``ollama pull gemma:2b``

In docker, it saved the results into ``/root/.ollama``

We're interested in the metadata about this model. You can find it in`` models/manifests/registry.ollama.ai/library``

What's the content of the file related to gemma?




In [1]:
!docker exec -it ollama bash -c "cat /root/.ollama/models/manifests/registry.ollama.ai/library/gemma/2b"

{"schemaVersion":2,"mediaType":"application/vnd.docker.distribution.manifest.v2+json","config":{"mediaType":"application/vnd.docker.container.image.v1+json","digest":"sha256:887433b89a901c156f7e6944442f3c9e57f3c55d6ed52042cbb7303aea994290","size":483},"layers":[{"mediaType":"application/vnd.ollama.image.model","digest":"sha256:c1864a5eb19305c40519da12cc543519e48a0697ecd30e15d5ac228644957d12","size":1678447520},{"mediaType":"application/vnd.ollama.image.license","digest":"sha256:097a36493f718248845233af1d3fefe7a303f864fae13bc31a3a9704229378ca","size":8433},{"mediaType":"application/vnd.ollama.image.template","digest":"sha256:109037bec39c0becc8221222ae23557559bc594290945a2c4221ab4f303b8871","size":136},{"mediaType":"application/vnd.ollama.image.params","digest":"sha256:22a838ceb7fb22755a3b0ae9b4eadde629d19be1f651f73efb8c6b4e2cd0eea0","size":84}]}

## Q3. Running the LLM
--------
Test the following prompt: "10 * 10". What's the answer?



In [7]:
from openai import OpenAI
import openai

# Set up the OpenAI client
openai.api_base = "http://localhost:11434/v1"
openai.api_key = "your-api-key"  # Replace with your actual API key if needed

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)


In [11]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gemma:2b',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [17]:
prompt = "10 * 10"

In [18]:
print(llm(prompt))

The answer is 100.

10 * 10 = 100.


Error: unknown command "eval" for "ollama"


## Q4. Donwloading the weights
------

We don't want to pull the weights every time we run a docker container. Let's do it once and have them available every time we start a container.

First, we will need to change how we run the container.

Instead of mapping the ``/root/.ollama`` folder to a named volume, let's map it to a local directory:

In [19]:
!mkdir ollama_files

!docker run -it \
    --rm \
    -v ./ollama_files:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama

mkdir: cannot create directory ‘ollama_files’: File exists
docker: Error response from daemon: Conflict. The container name "/ollama" is already in use by container "99f8781b3e0f0e65f9f3f7875f7faf5631acf542541e3660317f149346d442d1". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


Now pull the model:

In [ ]:
!docker exec -it ollama ollama pull gemma:2b #  in terminal

What's the size of the ``ollama_files/models`` folder?

 *   0.6G
 *  1.2G
 *  1.7G
 *  2.2G

Hint: on linux, you can use du -h for that.



In [23]:
!du -h  

1.6G	./ollama_files/models/blobs
8.0K	./ollama_files/models/manifests/registry.ollama.ai/library/gemma
12K	./ollama_files/models/manifests/registry.ollama.ai/library
16K	./ollama_files/models/manifests/registry.ollama.ai
20K	./ollama_files/models/manifests
1.6G	./ollama_files/models
1.6G	./ollama_files
40K	./.ipynb_checkpoints
1.6G	.


What's the size of the ollama_files/models folder?

 *   0.6G
 *  1.2G
 *  1.7G
 *  2.2G

Hint: on linux, you can use du -h for that.

## Q5. Adding the weights
-----
Let's now stop the container and add the weights to a new image

For that, let's create a ``Dockerfile:``

In [52]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gemma:2b',
        messages=[{"role": "user", "content": prompt},
                 ],
        temperature=0.0
    )
    print("Response:", response)
    print("Response:", response.choices[0].message.content)
    print("Completion Tokens:", response.usage.completion_tokens)
    
    return response.choices[0].message.content

In [53]:
prompt = "What's the formula for energy?"

In [54]:
output = llm(prompt)

Response: ChatCompletion(id='chatcmpl-270', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure, here's the formula for energy:\n\n**E = K + U**\n\nWhere:\n\n* **E** is the energy in joules (J)\n* **K** is the kinetic energy in joules (J)\n* **U** is the potential energy in joules (J)\n\n**Kinetic energy (K)** is the energy an object possesses when it moves or is in motion. It is calculated as half the product of an object's mass (m) and its velocity (v) squared:\n\n**K = 1/2 * m * v^2**\n\n**Potential energy (U)** is the energy an object possesses when it is in a position or has a specific configuration. It is calculated as the product of an object's mass and the gravitational constant (g) multiplied by the height or distance of the object from a reference point.\n\n**Gravitational potential energy (U)** is given by the formula:\n\n**U = mgh**\n\nWhere:\n\n* **m** is the mass of the object in kilograms (kg)\n* **g** is the accelera

In [ ]:
len(output)